## Sidenotes (definitions, code snippets, resources, etc.)
- Note on data structure: list
    - empty list has a truth value of false
- [Feature Selection with scikit-learn for intro_to_ml](http://napitupulu-jon.appspot.com/posts/feature-selection-ud120.html)
    - Looks very helpful for copying notes, course materials
    - Investigate meaning of `# %%writefile new_enron_feature.py` inserted at top of edited studentMain.py module

### Useful Python 2 Code Snippets
```python
### there can be many "to" emails, but only one "from", so the
### "to" processing needs to be a little more complicated
# uses counter for iterating through, duplicates process for cc_emails
#   does not seem very pythonic, but maybe clearest method
if to_emails:
    ctr = 0  # counter for iterating through, perhaps not pythonic
    while not to_poi and ctr < len(to_emails):
        if to_emails[ctr] in poi_email_list:
            to_poi = True
        ctr += 1
```

### Useful git code snippets
- `git reset --soft HEAD~`
    - Leaves working tree as it was before git commit


### Techniques


### IPython functions


## Why Feature Selection?
"Make everything as simple as possible, but no simpler" - Albert Einstein

Two major things aspects:
1. Select best features, leaving out unecessary data
- Adding new features to explore data, using intuition

### Coding up a new feature (with Enron data example)
Process that Katie uses:
- Use her human intuition
    - Quiz 1: POIs might send other POIs emails more often than the general population
    - Quiz 2: Feature scaling might provide more useful visualization
- Code up the new feature 
    - Quiz 1: Calculate aggregate of emails sent from POIs to each person
    - Quiz 2: 
- Visualize
    - Quiz 1: See scatter plot below to check whether feature gives discriminating power between POIs and non-POIs.
- Repeat
    - Improve preceeding parts of process
    - Zero in onn  the feature that would be most helpful
    
__Visualizations:__

_Quiz 1:_
![quiz1 scatterplot](lesson_11_images/quiz1_scatter.png)
